In [202]:
import numpy as np
np.random.seed(5)

import functions_class as fx
import classx as cl
import log_reg_functions as lrf
import loaddata as ld


import time
import matplotlib.pyplot as plt
import pandas as pd
import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss, f1_score


# A ) 
Load data

In [203]:
x,y = ld.load_data(scaler='minmax')

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size=0.25, random_state= 0, shuffle=True, stratify=y)

xtrain,xval,ytrain,yval = train_test_split(xtrain,ytrain, test_size=0.25, random_state= 0, shuffle=True, stratify=ytrain)
# do this because want our test to be seperate, first splitting into test and train, and then splitting traindata 
#into training and validation

nx_train, ny_train = xtrain.shape
nx_test, ny_test = xtest.shape



### f1-score is better when splitting data 50/50, this indicates that our algorithm learns more. The accuracy is expected to be lower for test data in this case. The dataset is biased, (read more) and therefore will give higher accuracy on biased datasets. 

In [204]:

indices = np.where(y == 1)
indices_zero = np.where(y == 0)
datapoints = np.random.choice(indices_zero[0], size=y[indices[0]].shape[0], replace=False)

x_new = np.vstack((x[indices[0],:],x[datapoints,:]))
y_new = np.vstack((y[indices[0]],y[datapoints]))

xtrain = x_new
ytrain = y_new

nx_train, ny_train = xtrain.shape
nx_test, ny_test = xtest.shape


# B) 
Egen logistic regression med gradient descent.


In [4]:
eta = 0.1 # learning rate
doplot = True
doprint = False
Niteration = 250
beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_val=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

start = time.time()
for iter in range(Niteration):
    
    sig = lrf.sigmoid(xtrain@beta)
    gradient = lrf.gradient_ols(xtrain,ytrain,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(xtrain@beta,ytrain.T)
    cost_val = lrf.cost_log_ols(xval@beta,yval.T) # do this for testdata at the end. 
    #Log Loss function from sklearn
    logloss=log_loss(ytrain, np.round(xtrain@beta), eps=1e-16, normalize=True)
    if doprint:
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_val)
    if doplot:
        costvec.append(cost.ravel())
        costvec_val.append(cost_val.ravel())
        loglvec.append(logloss)
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_val, 'r')
        #plt.plot(xaxis, loglvec, 'g')
        plt.pause(1e-12)
plt.show()    
end = time.time()
print(end - start)
   
# et relevant plot er iterativt plot. 

16.249404430389404


In [5]:
# making confusion matrix to check observed data with model predictions. 

predictions = xtest@beta
sig_val = lrf.sigmoid(predictions)
sig_val = np.round(sig_val)

cm = confusion_matrix(ytest , sig_val.astype(int))
print(cm)

[[4528 1313]
 [ 790  869]]


# Accuracy. 
Både egen kode og tester med scikit. 

In [6]:

activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')
print(f1_score(ytrain, classes))

activation =lrf.sigmoid(xval@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==yval)/len(activation),'% Validation Accuracy')
print(f1_score(yval, classes))


activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')
print(f1_score(ytest, classes))



64.70012055455094 % Training Accuracy
0.5965033158212041
72.05333333333333 % Validation Accuracy
0.44919411352487737
71.96 % Test Accuracy
0.4524863316844571


# Egen logistisk regresjon med stokastisk gradient descent

In [9]:
eta = 0.1 # learning rate
doplot = True
doprint = False
Niteration = 250
batch_size = 3000
batch_size_held_out = nx_train-batch_size

beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_test=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

indexes = np.arange(nx_train)


start = time.time()
for iter in range(Niteration):
    datapoints = np.random.choice(indexes, size=batch_size, replace=False)
    batch_x = xtrain[datapoints,:]
    batch_y = ytrain[datapoints]
    
    batch_x_held_out = np.delete(xtrain, datapoints, axis=0)
    batch_y_held_out = np.delete(ytrain, datapoints).reshape([batch_size_held_out,1])
    
    sig = lrf.sigmoid(batch_x@beta)
    gradient = lrf.gradient_ols(batch_x,batch_y,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(batch_x@beta,batch_y.T)
    cost_test = lrf.cost_log_ols(batch_x_held_out@beta,batch_y_held_out.T)
    #Log Loss function from sklearn
    if doprint:
        logloss=log_loss(batch_y, np.round(batch_x@beta), eps=1e-16, normalize=True)
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_test)
    if doplot:
        costvec.append(cost.ravel())
        costvec_test.append(cost_test.ravel())
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_test, 'r')
        plt.pause(1e-12)
plt.show()

end = time.time()
print(end - start)
 

21.221460819244385


In [10]:
# making confusion matrix to check observed data with model predictions. 

predictions = xtest@beta
sig_val = lrf.sigmoid(predictions)
sig_val = np.round(sig_val)

cm = confusion_matrix(ytest , sig_val.astype(int))
print(cm)

[[4382 1459]
 [ 732  927]]


In [192]:
activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')
print(f1_score(ytrain, classes))

activation =lrf.sigmoid(xval@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==yval)/len(activation),'% Validation Accuracy')
print(f1_score(yval, classes))


activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')
print(f1_score(ytest, classes))


70.47703703703704 % Training Accuracy
0.4567066521264994
71.43111111111111 % Validation Accuracy
0.4634390651085141
70.78666666666666 % Test Accuracy
0.45834363411619283


In [197]:
classes

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]])

In [12]:

model = LogisticRegression()
model.fit(xtrain, ytrain)
predicted_classes = model.predict(xtrain)
accuracy = accuracy_score(ytrain.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytrain, predicted_classes)

print(accuracy, '% Training Accuracy')

predicted_classes = model.predict(xtest)
accuracy = accuracy_score(ytest.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytest, predicted_classes)

print(accuracy, '% Test Accuracy')

C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


71.26280892103676 % Training Accuracy
77.36 % Test Accuracy


# C) 
Neural Network. 

In [112]:
#self.X_data_full = X_data 
#self.Y_data_full = Y_data

n_inputs, n_features = x.shape

#self.epochs = epochs
#self.batch_size = batch_size
#self.iterations = self.n_inputs // self.batch_size
eta = 0.001
lmbd = 10e-05


n_hidden_neurons=50  # velge andre verdier om vi vil
n_categories=2
epochs=100
batch_size=80
eta=0.01
n_hidden_neurons2=20  # velge andre verdier om vi vil



hidden_weights = np.random.randn(n_features, n_hidden_neurons)
hidden_weights2 = np.random.randn(n_hidden_neurons, n_hidden_neurons2)
hidden_bias = np.zeros(n_hidden_neurons) + 0.1
hidden_bias2 = np.zeros(n_hidden_neurons2) + 0.1

output_weights = np.random.randn(n_hidden_neurons2, n_categories)
output_bias = np.zeros(n_categories) + 0.1

In [113]:
print(hidden_weights.shape)
print(hidden_bias.shape)
print(hidden_weights2.shape)
print(hidden_bias2.shape)
print(output_weights.shape)
print(output_bias.shape)
print(x.shape)

(91, 50)
(50,)
(50, 20)
(20,)
(20, 2)
(2,)
(30000, 91)


In [114]:
z_h = np.matmul(x, hidden_weights) + hidden_bias
a_h = lrf.sigmoid(z_h)

z_h2 = np.matmul(a_h, hidden_weights2) + hidden_bias2
a_h2 = lrf.sigmoid(z_h2)

z_o = np.matmul(a_h2, output_weights) + output_bias

exp_term = np.exp(z_o)
probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

In [115]:
print(x.shape)
print(z_h.shape)
print(a_h.shape)
print(z_h2.shape)
print(a_h2.shape)
print(z_o.shape)
print(probabilities)

(30000, 91)
(30000, 50)
(30000, 50)
(30000, 20)
(30000, 20)
(30000, 2)
[[0.86973101 0.13026899]
 [0.18130462 0.81869538]
 [0.16498562 0.83501438]
 ...
 [0.94564342 0.05435658]
 [0.03933096 0.96066904]
 [0.38076984 0.61923016]]


In [120]:
error_output = probabilities - y
error_hidden2 = np.matmul(error_output, output_weights.T) * a_h2 * (1 - a_h2)
error_hidden = np.matmul(error_hidden2, hidden_weights2.T) * a_h * (1 - a_h) #a_h * (1 - a_h) is specific for sigmoid


output_weights_gradient = np.matmul(a_h2.T, error_output)
output_bias_gradient = np.sum(error_output, axis=0)

hidden_weights_gradient2 = np.matmul(a_h.T, error_hidden2)
hidden_bias_gradient2 = np.sum(error_hidden2, axis=0)

hidden_weights_gradient = np.matmul(x.T, error_hidden)
hidden_bias_gradient = np.sum(error_hidden, axis=0)

if lmbd > 0.0:
    output_weights_gradient += lmbd * output_weights
    hidden_weights_gradient2 += lmbd * hidden_weights2
    hidden_weights_gradient += lmbd * hidden_weights
    
output_weights -= eta * output_weights_gradient
output_bias -= eta * output_bias_gradient

hidden_weights2 -= eta * hidden_weights_gradient2
hidden_bias2 -= eta * hidden_bias_gradient2

hidden_weights -= eta * hidden_weights_gradient
hidden_bias -= eta * hidden_bias_gradient


z_h = np.matmul(x, hidden_weights) + hidden_bias
a_h = lrf.sigmoid(z_h)

z_h2 = np.matmul(a_h, hidden_weights2) + hidden_bias2
a_h2 = lrf.sigmoid(z_h2)

z_o = np.matmul(a_h2, output_weights) + output_bias

exp_term = np.exp(z_o)
probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)


C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


In [122]:
probabilities

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [227]:
Niter = 1000
x=xtrain
y=ytrain

n_inputs, n_features = x.shape

#self.epochs = epochs
#self.batch_size = batch_size
#self.iterations = self.n_inputs // self.batch_size
lmbd = 0.1


n_hidden_neurons=20  # velge andre verdier om vi vil
n_categories=2

eta=0.0001
n_hidden_neurons2=5  # velge andre verdier om vi vil



hidden_weights = np.random.randn(n_features, n_hidden_neurons)
hidden_weights2 = np.random.randn(n_hidden_neurons, n_hidden_neurons2)
hidden_bias = np.zeros(n_hidden_neurons) + 0.1
hidden_bias2 = np.zeros(n_hidden_neurons2) + 0.1

output_weights = np.random.randn(n_hidden_neurons2, n_categories)
output_bias = np.zeros(n_categories) + 0.1

for i in range(Niter):
    z_h = np.matmul(x, hidden_weights) + hidden_bias
    a_h = lrf.sigmoid(z_h)

    z_h2 = np.matmul(a_h, hidden_weights2) + hidden_bias2
    a_h2 = lrf.sigmoid(z_h2)

    z_o = np.matmul(a_h2, output_weights) + output_bias

    #probabilities = lrf.sigmoid(z_o)
    exp_term = np.exp(z_o)
    probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

    error_output = probabilities - y
    error_hidden2 = np.matmul(error_output, output_weights.T) * a_h2 * (1 - a_h2)
    error_hidden = np.matmul(error_hidden2, hidden_weights2.T) * a_h * (1 - a_h) #a_h * (1 - a_h) is specific for sigmoid


    output_weights_gradient = np.matmul(a_h2.T, error_output)
    output_bias_gradient = np.sum(error_output, axis=0)

    hidden_weights_gradient2 = np.matmul(a_h.T, error_hidden2)
    hidden_bias_gradient2 = np.sum(error_hidden2, axis=0)

    hidden_weights_gradient = np.matmul(x.T, error_hidden)
    hidden_bias_gradient = np.sum(error_hidden, axis=0)

    if lmbd > 0.0:
        output_weights_gradient += lmbd * output_weights
        hidden_weights_gradient2 += lmbd * hidden_weights2
        hidden_weights_gradient += lmbd * hidden_weights

    output_weights -= eta * output_weights_gradient
    output_bias -= eta * output_bias_gradient

    hidden_weights2 -= eta * hidden_weights_gradient2
    hidden_bias2 -= eta * hidden_bias_gradient2

    hidden_weights -= eta * hidden_weights_gradient
    hidden_bias -= eta * hidden_bias_gradient




In [228]:
#print(hidden_weights)


print((probabilities))
print(np.isnan(probabilities).sum())

[[0.50007771 0.49992229]
 [0.50008078 0.49991922]
 [0.50007771 0.49992229]
 ...
 [0.50001634 0.49998366]
 [0.50001632 0.49998368]
 [0.50001632 0.49998368]]
0


In [229]:

classes=np.argmax(probabilities,axis=1)
print(100*np.sum(classes==ytrain.T)/len(ytrain),'% Training Accuracy')
print(f1_score(ytrain, classes))

51.25828812537673 % Training Accuracy
0.07519656897784131


In [217]:
classes=np.argmax(probabilities,axis=1)
print(classes)
ytrain.T

[0 0 0 ... 0 0 0]


array([[1, 1, 1, ..., 0, 0, 0]], dtype=int64)